In [ ]:
!pip install pytorch-tabnet
import pandas as pd
import numpy  as np
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.model_selection import KFold 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_data = pd.read_csv('Table 1.csv')
test_data  = pd.read_csv('Table 2.csv') 
sample     = pd.read_csv('Table 1.csv') 
solution   = pd.read_csv('Table 1.csv') 

In [ ]:
features = ['order_id', 'product_id', 'category_id', 'price']

In [ ]:
X      = train_data[features]
y      = np.log1p(train_data["order_id"])
X_test = test_data[features]
y_true = solution["order_id"]

In [ ]:
X      =      X.apply(lambda x: x.fillna(x.mean()),axis=0)
X_test = X_test.apply(lambda x: x.fillna(x.mean()),axis=0)

In [ ]:
X      = X.to_numpy()
y      = y.to_numpy().reshape(-1, 1)
X_test = X_test.to_numpy()

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
predictions_array =[]
CV_score_array    =[]
for train_index, test_index in kf.split(X):
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    regressor = TabNetRegressor(verbose=0,seed=42)
    regressor.fit(X_train=X_train, y_train=y_train,
              eval_set=[(X_valid, y_valid)],
              patience=100, max_epochs=400,
              eval_metric=['rmse'])
    CV_score_array.append(regressor.best_cost)
    predictions_array.append(np.expm1(regressor.predict(X_test)))

predictions = np.mean(predictions_array,axis=0)
with open('file.csv', 'w') as f:
    for i in range(len(predictions)):
        f.write(str(predictions[i]))

# CV_score_array = [0.029263964438688807, 0.037366541157093786, 0.04662104266543657, 0.019538891973906628, 0.05033337924673994]

Stop training because you reached max_epochs = 400 with best_epoch = 338 and best_val_0_rmse = 0.02926


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 400 with best_epoch = 367 and best_val_0_rmse = 0.03737


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 400 with best_epoch = 303 and best_val_0_rmse = 0.04662


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Stop training because you reached max_epochs = 400 with best_epoch = 379 and best_val_0_rmse = 0.01954


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)



Early stopping occurred at epoch 379 with best_epoch = 279 and best_val_0_rmse = 0.05033


/usr/local/lib/python3.9/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Перекрестная проверка (CV score) в основном используется в прикладном машинном обучении для оценки навыков модели машинного обучения на невидимых данных. То есть используется ограниченная выборка, для оценки ожидаемой работы модели в целом при использовании для прогнозирования данных, не используемых во время обучения модели

In [ ]:
print("The CV оценка составляет %.5f" % np.mean(CV_score_array,axis=0) )       

The CV оценка составляет 0.03662


В контексте Kaggle LB score означает LeaderBoard (выделено мной)

In [ ]:
from sklearn.metrics import mean_squared_log_error 
RMSLE = np.sqrt( mean_squared_log_error(y_true, predictions)) 
print("The LB score is %.5f" % RMSLE ) 

The LB score is 0.02979


In [ ]:
predictions = open('file.csv')
predictions = predictions.read()

predictions = predictions.replace("[","")
predictions = predictions.split("]")
predictions.pop(len(predictions)-1)
for i in range(len(predictions)):
    predictions[i] = float(predictions[i])
    predictions[i] = np.asarray(predictions[i])
# df_new = predictions[np.isfinite(predictions).all(1)]

In [ ]:
# sample.iloc[:,1:] = predictions
# sample.to_csv('kzcopy2.csv',index=False) 
output = pd.DataFrame(predictions)
output.to_csv('kzcopy2.csv', index=False)